# PCA Regression

The goal for this notebook is to give you three baseline models 

In [133]:
import numpy as np
import pandas as pd
import logging
import anndata as ad


logging.basicConfig(level=logging.INFO)

In [134]:
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt

In [5]:
import logging
import anndata as ad

from scipy.sparse import csc_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression

logging.basicConfig(level=logging.INFO)

In [12]:
Y_train = ad.read_h5ad("Adt_processed_training.h5ad") #gex is gene expression which are RNA
Y_test = ad.read_h5ad("Adt_processed_testing.h5ad") #gex is gene expression which are RNA
X_train = ad.read_h5ad("Gex_processed_training.h5ad") # adt is protein
X_test = ad.read_h5ad("Gex_processed_testing.h5ad") # adt is protein


### RMSE Metric

The metric for task 1 is RMSE on the `adata.X` data.

In [32]:
def calculate_rmse(true_test_mod2, pred_test_mod2):
    return  mean_squared_error(true_test_mod2.X.toarray(), pred_test_mod2.X, squared=False) # .toarray will turn it to numpy array


### Method

Let's try a method that runs linear regression on PCA transformed data before projecting the data back to the feature space.

In [132]:
def baseline_PC_regression(X_train, Y_train, X_test):
    '''Baseline method training a linear regressor on the input data'''
    X_joint = ad.concat(
        {"train": X_train, "test": X_test},
        axis = 0,
        join = "outer",
        label = "group",
        fill_value = 0,
        index_unique = "-", 
    )
    
    # Do PCA on the input data
    logging.info('Performing dimensionality reduction on RNA values...')
    embedder_RNA = TruncatedSVD(n_components = 50)
    X_pca = embedder_RNA.fit_transform(X_joint.X)
    

    
    # split dimred mod 1 back up for training
    PC_train = X_pca[X_joint.obs['group'] == 'train']
    PC_test = X_pca[X_joint.obs['group'] == 'test']
    y_train = Y_train.X.toarray()
    
    assert len(PC_train) + len(PC_test) == len(X_pca)
    
    logging.info('Running Linear regression...')
    
    reg = LinearRegression()
    
    # Train the model on the PCA reduced modality 1 and 2 data
    reg.fit(PC_train, y_train)
    y_pred = reg.predict(PC_test)
    
    # Project the predictions back to the modality 2 feature space
    
    pred_test_mod2 = ad.AnnData(
        X = y_pred,
        obs = X_test.obs,
        var = Y_train.var
    )
    
    # Add the name of the method to the result
    pred_test_mod2.uns["method"] = "linear"
    
    return pred_test_mod2

In [131]:
def baseline_PC_regression2(X_train, Y_train, X_test):
    '''Baseline method training a linear regressor on the input data'''
    X_tr = X_train.X.toarray()
    X_bar = np.mean(X_tr, axis = 0)
    X_sd = np.std(X_tr, axis=0)
    X_te = X_test.X.toarray()
    X_tr1 = (X_tr - X_bar) / X_sd # center and normalized to have variance 1
    X_te1 = (X_te - X_bar) / X_sd # Normalized testing data
    
    
    # Do PCA on the X_train data
    logging.info('Performing dimensionality reduction on X_train RNA values...')
    svd = TruncatedSVD(n_components = 50)
    svd.fit(X_tr1)
    singular_vectors = svd.components_
    PC_train = np.dot(X_tr1, singular_vectors.T)
    PC_test = np.dot(X_te1, singular_vectors.T)
        
    y_train = Y_train.X.toarray()
    
    
    logging.info('Running Linear regression...')
    
    reg = LinearRegression()
    
    # Train the model on the PCA reduced modality 1 and 2 data
    reg.fit(PC_train, y_train)
    y_pred = reg.predict(PC_test)
    
    # Project the predictions back to the modality 2 feature space
    
    pred_Y = ad.AnnData(
        X = y_pred,
        obs = X_test.obs,
        var = Y_train.var
    )
    
    # Add the name of the method to the result
    pred_Y.uns["method"] = "linear"
    
    return pred_Y

Now, for comparison, let's create a simple dummy method that simply returns the mean for the input modality 2 data. This method returns an identical prediction for all cells and ignores the modality 1 information altogether.

In [62]:
def baseline_mean(input_train_mod1, input_train_mod2, input_test_mod1):
    '''Dummy method that predicts mean(input_train_mod2) for all cells'''
    logging.info('Calculate mean of the training data modality 2...')
    y_pred = np.repeat(input_train_mod2.X.mean(axis=0).reshape(-1,1).T, input_test_mod1.shape[0], axis=0)
    y_pred = np.array(y_pred)
    # Prepare the ouput data object
    pred_test_mod2 = ad.AnnData(
        X = y_pred,
        obs=input_test_mod1.obs,
        var=input_train_mod2.var,
    )
    
    pred_test_mod2.uns["method"] = "mean"

    return pred_test_mod2

#### Run comparison

Let's run the simple and dummy method side-by-side and compare performance.

In [66]:
for method in [baseline_PC_regression, baseline_mean]:
    # Run prediction
    pred_Y = method(X_train, Y_train, X_test)
    # Calculate RMSE
    rmse = calculate_rmse(Y_test, pred_Y)
    # Print results
    print(f'{pred_Y.uns["method"]} had a RMSE of {rmse:.4f}')

INFO:root:Performing dimensionality reduction on RNA values...
INFO:root:Running Linear regression...
INFO:root:Calculate mean of the training data modality 2...


linear had a RMSE of 0.3821
mean had a RMSE of 0.5487


In [130]:
for method in [baseline_PC_regression2, baseline_mean]:
    # Run prediction
    pred_Y = method(X_train, Y_train, X_test)
    # Calculate RMSE
    rmse = calculate_rmse(Y_test, pred_Y)
    # Print results
    print(f'{pred_Y.uns["method"]} had a RMSE of {rmse:.4f}')

INFO:root:Performing dimensionality reduction on X_train RNA values...
INFO:root:Running Linear regression...
INFO:root:Calculate mean of the training data modality 2...


linear had a RMSE of 0.3711
mean had a RMSE of 0.5487


As expected, the linear model does better than the dummy method. Now the challenge is up to you! Can you do better than this baseline?